In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

### Implementação do Zscore

In [2]:
#Código do Zscore que fiz na lista 02, vou utilizar ele para normalizar os dados
class Zscore():
    def __init__(self, columnNumber = 2):
        self.__means = np.empty(columnNumber)
        self.__stds = np.empty(columnNumber)
        self.__quantity = 0
        
    def __setMeans(self, newMeans):
        self.__means = newMeans
    
    def getMeans(self):
        return self.__means
    
    def __setStds(self, newStds):
        self.__stds = newStds
    
    def getStds(self):
        return self.__stds
    
    def __setQuantity(self, newQ):
        self.__quantity = newQ
    
    def getQuantity(self):
        return self.__quantity
    
    def __addValues(self, mu, sigma):
        means = self.getMeans()
        stds = self.getStds()
        quantity = self.getQuantity()
        
        means[quantity] = mu
        stds[quantity] = sigma
        
        self.__setMeans(means)
        self.__setStds(stds)
        
        self.__setQuantity(quantity + 1)
    
    def scale(self, data):
        rows = data.shape[0]
        columns = data.shape[1]
        #Utiliza a Normalização Z-score, seria o equivalente ao Standard Scaler
        #Recebe um conjunto de dados e Retorna o mesmo conjunto de dados normalizado com média 0 e dp 1
        dataScaled = np.empty([rows, 0])
        for i in range(columns):
            #Esse método faz a normalização coluna por coluna, onde i é o número da coluna
            dataColumn = data[:, [i]]

            #Cálculo da média e desvio-padrão da coluna que vai ser normalizada
            mu = np.mean(dataColumn)
            sigma = np.std(dataColumn)

            columnScaled = (dataColumn - mu)/sigma
            #columnScaled = (xi - mu)/sigma
            #operação broadcasting para toda a coluna
            dataScaled = np.c_[dataScaled, columnScaled]
            
            self.__addValues(mu, sigma)
            #adiciona a coluna no dataset normalizado
        #print(dataScaled)
        return dataScaled
    
    def unscale(self, data, column = -1):
        # pensando em implementar um atributo dataset original
        # mas esse método tem como função principal fazer o "unscale" de novos atributos de um dado escalado
        # anteriormente, como por exemplo ŷ que é escalado na normal com base nos dados de y
        if column == -1:
            mu = self.getMeans()
            sigma = self.getStds()
            
        elif column >= self.getQuantity():
            print("ERRO: Número de Coluna ", column ," Inválida.")
            return
            
        else:
            mu = self.getMeans()[column]
            sigma = self.getStds()[column]
            
        return sigma * data + mu
        

## Implementação do KNN

In [3]:
class KNearestNeighbours():
    def calculateDistance(self, point):

        if self.distance == "euclidiana":
            return np.apply_along_axis(lambda x: np.linalg.norm(x - point), axis=1, arr=self.X)
        
        elif self.distance == "mahalanobis": # aparentemente tá correto
            x_diff = self.X - point
            x_cov = np.linalg.inv(self.cov)
            x_diff_cov = np.dot(x_diff, x_cov)
            x_diff_cov_norm = np.sqrt(np.sum(x_diff_cov * x_diff, axis=1))
            
            return x_diff_cov_norm
        else:
            return -1
        
    def select_KNN(self, datapoint):
        dists = self.calculateDistance(datapoint)
        
        distsIdx = np.argsort(dists)[:self.K]
        unique, counts = np.unique(self.y[distsIdx], return_counts=True)
        
        idx = np.argsort(counts)[::-1]
        
        sorted_unique = unique[idx]
        sorted_counts = counts[idx]
        
        return sorted_unique[0]

    def train(self, X, y, K = 3,distance = "euclidean"):
        self.X = X
        self.y = y
        self.K = K
        self.distance = distance
        self.cov = np.array(np.cov(X.T))
        self.cov = self.cov + 1e-10  * np.identity(self.cov.shape[0])
        
    def predict(self, X):
        y_pred = np.apply_along_axis(self.select_KNN, 1, X) 
        # retorna uma matriz onde as linhas são os K menores distancias para cada valor em X,
        # a matriz fica tamanho de linhas de X por K colunas
        
        # aparentemente Ok
        return y_pred
   

## Aplicação dos Modelos e Métricas

Kfold que será utilizado na avaliação dos modelos

In [4]:
def kfold(X): # mesmo kfold que utilizei na lista 02
    kfolds = []
    n_splits = 10

    # Embaralhar os índices dos dados
    indices = np.random.permutation(len(X))

    # Dividir os índices em n_splits partes iguais
    folds_indices = np.array_split(indices, n_splits)

    # Gerar KFold para cada conjunto de índices
    for i in range(n_splits):
        train_indices = np.concatenate(folds_indices[:i] + folds_indices[i+1:])
        test_indices = folds_indices[i]
        kfolds.append((train_indices, test_indices))
    
    return train_indices, test_indices, kfolds

### Métricas
Abaixo estão as funções que farão os cálculos das métricas solicitadas.

In [5]:
def accuracy_score(y_true, y_pred):
    # Accuracy Score: a proporção de acertos entre as previsões (y_pred) e os valores verdadeiros (y_true).

    return np.mean(y_true == y_pred)

def recall_score(y_true, y_pred):
    # Recall Score: a proporção de verdadeiros positivos (TP) sobre o total de casos positivos (TP + FN).

    TP = np.sum((y_true == 1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    return TP / (TP + FN)

def precision_score(y_true, y_pred):
    # Precision Score: a proporção de verdadeiros positivos (TP) 
    # sobre o total de casos classificados como positivos (TP + FP).

    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    return TP / (TP + FP)

def f1_score(y_true, y_pred):
    # F1 Score: a média harmônica entre o Precision Score e o Recall Score.
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall)

### Código que fará a avaliação do KNN

In [6]:
def questao1():
    dataset = np.genfromtxt('.//ama_lista_03//kc2.csv', delimiter=',')
    normalizer = Zscore(dataset.shape[1])
    X = dataset[:, :21]
    Y = dataset[:, 21]
    X = normalizer.scale(X)
    
    train_indices, test_indices, kfolds = kfold(X)
    
    possibilidades = [(1, "euclidiana"), (5, "euclidiana"), (1, "mahalanobis"), (5, "mahalanobis")]
    
    for p in possibilidades:
        K = p[0]
        D = p[1]
        
        acc = np.zeros(10)
        rcl = np.zeros(10)
        prc = np.zeros(10)
        f1  = np.zeros(10)
    
        for i, (train_indices, test_indices) in enumerate(kfolds): 
            KNN = KNearestNeighbours()

            X_train = X[train_indices, :]
            Y_train = Y[train_indices]

            X_test = X[test_indices, :]
            Y_test = Y[test_indices]
            
            KNN.train(X_train, Y_train, K, D)
            
            Y_hat = KNN.predict(X_test)
            
            acc[i] =(accuracy_score(Y_test, Y_hat))
            rcl[i] =(recall_score(Y_test, Y_hat))
            prc[i] =(precision_score(Y_test, Y_hat))
            f1[i]  =(f1_score(Y_test, Y_hat))
        
        
        print("Métricas do KNN para K =", K, " e Distância do tipo", D)
        print("            Médias            Desvio-Padrão")
            
        print("Acurácia:  ", f"{acc.mean():.15f}", f"{acc.std():.15f}")
        print("Revocação: ", f"{rcl.mean():.15f}", f"{rcl.std():.15f}")
        print("Precisão:  ", f"{prc.mean():.15f}", f"{prc.std():.15f}")
        print("F1-Score:  ", f"{f1.mean():.15f}", f"{f1.std():.15f}")
            
        print("_"*60)
        print()
            
    

### Código que fará a avaliação da Árvore de Decisão

In [7]:
def questao2():
    dataset = np.genfromtxt('.//ama_lista_03//kc2.csv', delimiter=',')
    normalizer = Zscore(dataset.shape[1])
    X = dataset[:, :21]
    Y = dataset[:, 21]
    X = normalizer.scale(X)
    
    train_indices, test_indices, kfolds = kfold(X)
    
    possibilidades = ["gini", "entropy"]
    
    for p in possibilidades:
        tipo = p
        
        acc = np.zeros(10)
        rcl = np.zeros(10)
        prc = np.zeros(10)
        f1  = np.zeros(10)
    
        for i, (train_indices, test_indices) in enumerate(kfolds): 
            DTC = DecisionTreeClassifier(random_state = 42, criterion = p)

            X_train = X[train_indices, :]
            Y_train = Y[train_indices]

            X_test = X[test_indices, :]
            Y_test = Y[test_indices]
            
            DTC.fit(X_train, Y_train)
            
            Y_hat = DTC.predict(X_test)
            
            acc[i] =(accuracy_score(Y_test, Y_hat))
            rcl[i] =(recall_score(Y_test, Y_hat))
            prc[i] =(precision_score(Y_test, Y_hat))
            f1[i]  =(f1_score(Y_test, Y_hat))
        
        
        print("Métricas da Árvore de Decisão construída a partir de", p)
        print("            Médias            Desvio-Padrão")
            
        print("Acurácia:  ", f"{acc.mean():.15f}", f"{acc.std():.15f}")
        print("Revocação: ", f"{rcl.mean():.15f}", f"{rcl.std():.15f}")
        print("Precisão:  ", f"{prc.mean():.15f}", f"{prc.std():.15f}")
        print("F1-Score:  ", f"{f1.mean():.15f}", f"{f1.std():.15f}")
            
        print("_"*60)
        print()

## Final
Abaixo estão os resultados das médias e desvio-padrão de cada métrica

In [8]:
questao1()

Métricas do KNN para K = 1  e Distância do tipo euclidiana
            Médias            Desvio-Padrão
Acurácia:   0.733549783549784 0.052365297649957
Revocação:  0.731182151182151 0.185041982353849
Precisão:   0.724545666751549 0.141959240898355
F1-Score:   0.713456522613206 0.127312083332558
____________________________________________________________

Métricas do KNN para K = 5  e Distância do tipo euclidiana
            Médias            Desvio-Padrão
Acurácia:   0.790259740259740 0.071876139430455
Revocação:  0.786273726273726 0.112578972569190
Precisão:   0.785021645021645 0.081387041623325
F1-Score:   0.783026059985227 0.089499887940080
____________________________________________________________

Métricas do KNN para K = 1  e Distância do tipo mahalanobis
            Médias            Desvio-Padrão
Acurácia:   0.682034632034632 0.071928266563420
Revocação:  0.674431401931402 0.109243051293509
Precisão:   0.699380480630481 0.135526064665932
F1-Score:   0.673063886755570 0.070269

In [9]:
questao2()

Métricas da Árvore de Decisão construída a partir de gini
            Médias            Desvio-Padrão
Acurácia:   0.733766233766234 0.071801466628448
Revocação:  0.751188811188811 0.135541780528282
Precisão:   0.724253246753247 0.170950651986556
F1-Score:   0.725787143613231 0.117414863462449
____________________________________________________________

Métricas da Árvore de Decisão construída a partir de entropy
            Médias            Desvio-Padrão
Acurácia:   0.706493506493507 0.091894270148445
Revocação:  0.668840326340326 0.121234437524527
Precisão:   0.731313131313131 0.192821509900049
F1-Score:   0.684328224653706 0.132154698835318
____________________________________________________________

